In [1]:
import os
import json
import pandas as pd
import traceback
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
load_dotenv()

True

In [2]:
KEY=os.getenv("OPENAI_API_KEY")

In [3]:
llm=ChatOpenAI(openai_api_key=KEY,model_name="gpt-3.5-turbo", temperature=0.5)

In [4]:
print(llm)


client=<openai.resources.chat.completions.Completions object at 0x12dc8fb20> async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x12dcc9240> temperature=0.5 openai_api_key=SecretStr('**********') openai_proxy=''


In [5]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2


In [6]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [7]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz {number} multiple choice questions for {subject} students in {tone} tone.
Make sure the questions are not repeated and check all the questions to be conforming the test as well.
Make sure to format your response like RESPONSE_KJSON below and use it as a guide. \
Ensure tomake {number} MCQs
### RESPONSE_JSON
{response_json}

"""

In [8]:
quiz_generation_prompt = PromptTemplate(
input_variables=["text", "number", "subject", "tone", "response_json"],
template=TEMPLATE)

In [9]:
quiz_chain=LLMChain(llm=llm,prompt=quiz_generation_prompt,output_key="quiz", verbose=True)

In [10]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""


In [11]:
quiz_evaluation_prompt=PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE2)


In [12]:
review_chain=LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)


In [13]:
generate_evaluate_chain=SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text", "number", "subject", "tone", "response_json"],
                                        output_variables=["quiz", "review"], verbose=True,)


In [14]:
file_path="/Users/vishypoosala/Documents/hacks/genai/mcq/data.txt"


In [15]:
file_path


'/Users/vishypoosala/Documents/hacks/genai/mcq/data.txt'

In [16]:
with open(file_path, 'r') as file:
    TEXT = file.read()

In [17]:
TEXT

"The term machine learning was coined in 1959 by Arthur Samuel, an IBM employee and pioneer in the field of computer gaming and artificial intelligence.[9][10] The synonym self-teaching computers was also used in this time period.[11][12]\n\nAlthough the earliest machine learning model was introduced in the 1950s when Arthur Samuel invented a program that calculated the winning chance in checkers for each side, the history of machine learning roots back to decades of human desire and effort to study human cognitive processes.[13] In 1949, Canadian psychologist Donald Hebb published the book The Organization of Behavior, in which he introduced a theoretical neural structure formed by certain interactions among nerve cells.[14] Hebb's model of neurons interacting with one another set a groundwork for how AIs and machine learning algorithms work under nodes, or artificial neurons used by computers to communicate data.[13] Other researchers who have studied human cognitive systems contribu

In [18]:
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [19]:
NUMBER=5 
SUBJECT="machine learning"
TONE="simple"

In [20]:
with get_openai_callback() as cb:
    response=generate_evaluate_chain(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject":SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
        )

/Users/vishypoosala/miniconda3/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:The term machine learning was coined in 1959 by Arthur Samuel, an IBM employee and pioneer in the field of computer gaming and artificial intelligence.[9][10] The synonym self-teaching computers was also used in this time period.[11][12]

Although the earliest machine learning model was introduced in the 1950s when Arthur Samuel invented a program that calculated the winning chance in checkers for each side, the history of machine learning roots back to decades of human desire and effort to study human cognitive processes.[13] In 1949, Canadian psychologist Donald Hebb published the book The Organization of Behavior, in which he introduced a theoretical neural structure formed by certain interactions among nerve cells.[14] Hebb's model of neurons interacting with one another set a groundwork for how AIs and machine learning algorithms work under nodes, or artificial neurons used 

In [21]:
response

{'text': "The term machine learning was coined in 1959 by Arthur Samuel, an IBM employee and pioneer in the field of computer gaming and artificial intelligence.[9][10] The synonym self-teaching computers was also used in this time period.[11][12]\n\nAlthough the earliest machine learning model was introduced in the 1950s when Arthur Samuel invented a program that calculated the winning chance in checkers for each side, the history of machine learning roots back to decades of human desire and effort to study human cognitive processes.[13] In 1949, Canadian psychologist Donald Hebb published the book The Organization of Behavior, in which he introduced a theoretical neural structure formed by certain interactions among nerve cells.[14] Hebb's model of neurons interacting with one another set a groundwork for how AIs and machine learning algorithms work under nodes, or artificial neurons used by computers to communicate data.[13] Other researchers who have studied human cognitive systems

In [22]:
print(cb.total_tokens)

1517


In [23]:
print(cb.prompt_tokens)

1022


In [25]:
print(cb.total_cost)

0.002523


In [36]:
quiz=response.get("quiz")[17:]
print(quiz)
quiz=json.loads(quiz)


{
    "1": {
        "mcq": "Who coined the term 'machine learning' in 1959?",
        "options": {
            "a": "Donald Hebb",
            "b": "Walter Pitts",
            "c": "Arthur Samuel",
            "d": "Warren McCulloch"
        },
        "correct": "c"
    },
    "2": {
        "mcq": "What was the earliest machine learning model introduced by Arthur Samuel in the 1950s?",
        "options": {
            "a": "Program for calculating winning chance in chess",
            "b": "Program for calculating winning chance in checkers",
            "c": "Program for calculating winning chance in tic-tac-toe",
            "d": "Program for calculating winning chance in poker"
        },
        "correct": "b"
    },
    "3": {
        "mcq": "Who introduced a theoretical neural structure in the book 'The Organization of Behavior'?",
        "options": {
            "a": "Arthur Samuel",
            "b": "Donald Hebb",
            "c": "Walter Pitts",
            "d": "Warren M

In [37]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [38]:
quiz_table_data

[{'MCQ': "Who coined the term 'machine learning' in 1959?",
  'Choices': 'a: Donald Hebb | b: Walter Pitts | c: Arthur Samuel | d: Warren McCulloch',
  'Correct': 'c'},
 {'MCQ': 'What was the earliest machine learning model introduced by Arthur Samuel in the 1950s?',
  'Choices': 'a: Program for calculating winning chance in chess | b: Program for calculating winning chance in checkers | c: Program for calculating winning chance in tic-tac-toe | d: Program for calculating winning chance in poker',
  'Correct': 'b'},
 {'MCQ': "Who introduced a theoretical neural structure in the book 'The Organization of Behavior'?",
  'Choices': 'a: Arthur Samuel | b: Donald Hebb | c: Walter Pitts | d: Warren McCulloch',
  'Correct': 'b'},
 {'MCQ': "What did Hebb's model of neurons interacting with one another set a groundwork for in machine learning?",
  'Choices': 'a: Programming languages | b: Artificial Intelligence | c: Neural networks | d: Quantum computing',
  'Correct': 'c'},
 {'MCQ': 'Who prop

In [40]:
quiz = pd.DataFrame(quiz_table_data)

In [41]:
quiz


,MCQ,Choices,Correct
0,Who coined the term 'machine learning' in 1959?,a: Donald Hebb | b: Walter Pitts | c: Arthur S...,c
1,What was the earliest machine learning model i...,a: Program for calculating winning chance in c...,b
2,Who introduced a theoretical neural structure ...,a: Arthur Samuel | b: Donald Hebb | c: Walter ...,b
3,What did Hebb's model of neurons interacting w...,a: Programming languages | b: Artificial Intel...,c
4,Who proposed the early mathematical models of ...,a: Donald Hebb | b: Walter Pitts | c: Arthur S...,b


In [42]:
quiz.to_csv("machinelearning.csv",index=False)